In [30]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, f_regression, mutual_info_regression


In [31]:
# data set from jay divided into two files
attributes = "Updated_2018_Dataset.csv"
attributes_df = pd.read_csv(attributes)

targets = "2018_Targets_2017.csv"
#targets dataframe contains all groups including Food 17
targets_df = pd.read_csv(targets)

attributes_df.info()
targets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Columns: 280 entries, Timestamp to Average of MORTGAGE30US
dtypes: float64(278), object(2)
memory usage: 881.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 23 columns):
Timestamp                     398 non-null int64
Food 17                       398 non-null float64
restaurants 17                398 non-null float64
Vegetables                    398 non-null float64
Other                         398 non-null float64
Meat                          398 non-null float64
Dairy products and eggs       398 non-null float64
Bakery                        398 non-null float64
Fruit                         398 non-null float64
All-items                     398 non-null float64
Food purchased from stores    398 non-null float64
Fresh or frozen beef          398 non-null float64
Fresh or frozen pork          398 non-null float64
Fresh or frozen chicken       398 non-null floa

In [32]:
#magic numbers to start at 1985 January and end December 2017
attributes_df = attributes_df[170:-6]
targets_df= targets_df[170:-2]

In [33]:
#individual target dataframes
CPI_df = targets_df['Food 17']
restaurants_df = targets_df['restaurants 17']
vegetables_df = targets_df['Vegetables ']
other_df = targets_df['Other ']
meat_df = targets_df['Meat']
dairyandeggs_df = targets_df['Dairy products and eggs']
bakery_df = targets_df['Bakery']
fruit_df = targets_df['Fruit']
allitems_df = targets_df['All-items']
fromstores_df = targets_df['Food purchased from stores']
beef_df = targets_df['Fresh or frozen beef']
pork_df = targets_df['Fresh or frozen pork']
chicken_df = targets_df['Fresh or frozen chicken']
dairy_df = targets_df['Dairy products']
eggs_df = targets_df['Eggs']
coffee_df = targets_df['Coffee']
babyfood_df = targets_df['Baby foods']
shelter18_df = targets_df['Shelter 18']
transportation_df = targets_df['Transportation']
gas_df = targets_df['Gasoline']
energy25_df = targets_df['Energy 25']
fishandseafood_df = targets_df['Fish seafood']

In [34]:
timestamps_df = targets_df['Timestamp']

food17_target_df =  CPI_df

# remoe the timestamps from both
targets_df = targets_df.drop('Timestamp', axis = 1)
attributes_df = attributes_df.drop('Timestamp', axis = 1)

# drop all attributes that are missing values for now
# revise this later when we predict them.
attributes_df =attributes_df.dropna(how='any', axis = 1)
att_df= attributes_df


In [35]:
#switches 
restaurants = True
vegetables = True
other = True
meat = True
dairyandeggs = True
bakery = True
fruit = True
allitems = True
fromstores = True
beef = True
pork = True
chicken = True
dairy = True
eggs = True
coffee = True
babyfood = True
shelter = True
transportation = True
gas = True
energy = True
fishandseafood = True


In [36]:
    attributes_df=att_df
    selector = SelectKBest(mutual_info_regression,k=29 )
    attributes_df = selector.fit_transform(attributes_df,food17_target_df)
    
    idxs_selected = selector.get_support(indices=True)
    # Create new dataframe with only desired columns, or overwrite existing
    features_dataframe_new = att_df[idxs_selected]
    print("food 17:")
    print(features_dataframe_new.info())
    
    oneyear = -12
    #print(attributes_df)
    
    #2017 data added to targets:
    att_train_oneyear_df = attributes_df[:-12]
    #target groupings
    tar_train_oneyear__df = targets_df[:oneyear]
    tar_test_oneyear_df = targets_df[oneyear:]
    # food 17 alone
    food17_train_oneyear_df =food17_target_df[:oneyear]
    food17_test_oneyear_df =food17_target_df[oneyear:]
    
    # Create linear regression object for one year
    regr_oneyear = linear_model.LinearRegression()
    # Train the model using the training sets
    regr_oneyear.fit(att_train_oneyear_df, food17_train_oneyear_df)
    # Make predictions using the testing set
    food17_pred_oneyear = regr_oneyear.predict(att_train_oneyear_df[-12:])
    
    # The mean absolute error
    print("Mean absolute error: %.2f"
          % mean_absolute_error(food17_test_oneyear_df, food17_pred_oneyear))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(food17_test_oneyear_df, food17_pred_oneyear))
    
    #target_CPI_pred.plot()
    lastyear = timestamps_df[oneyear:]
    lastyear= lastyear.as_matrix()
    
    # Plot outputs
    plt.scatter(lastyear,food17_test_oneyear_df,  color='black')
    plt.plot(lastyear,food17_pred_oneyear, color='blue', linewidth=3)
    #plt.axis([lastyear[0]-3,lastyear[11]+3,135,148])
    plt.xticks(())
    plt.yticks(())
    plt.ylabel('$')
    plt.xlabel('Months')
    #need to write script to compute dates from timestamps!
    plt.title('2017-01-01 to 2018-01-01')
    plt.show()

ValueError: could not convert string to float: '.'